Reshma a/p Ganesan (IS01082523) and Yasmine Essam (IS01082525)

In [9]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import capsolver  # Import CapSolver

# Set CapSolver API key
capsolver.api_key = "YOUR_CAPSOLVER_API_KEY"  # Replace with your actual API key

# Setup undetectable Chrome WebDriver
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

driver = uc.Chrome(options=options)
driver.get("https://www.lazada.com.my/products/sweet-mint-eyeliner-silkworm-pen-glitter-nude-bright-waterproof-double-head-eyeliner-pen-eye-makeup-i3366129750-s18081596663.html")
time.sleep(random.uniform(5, 10))  # Random delay to mimic human behavior

reviews_data = []
max_pages = 5  # Number of pages to scrape

# Function to solve CAPTCHA using CapSolver
def solve_captcha():
    print("CAPTCHA detected. Attempting to solve using CapSolver...")
    try:
        solution = capsolver.solve({
            "type": "ReCaptchaV2TaskProxyless",
            "websiteURL": driver.current_url,
            "websiteKey": "SITE_KEY_HERE"  # Replace with actual site key if known
        })
        print(f"CAPTCHA solved: {solution}")
        time.sleep(10)  # Allow Lazada to verify the CAPTCHA solution
        return solution
    except Exception as e:
        print(f"Failed to solve CAPTCHA: {e}")
        return None

# Scrape multiple pages
for page in range(1, max_pages + 1):
    print(f"Scraping Page {page}...")

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(5, 10))  # Random delay

    # Find all review elements
    reviews = driver.find_elements(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[1]/div')

    if not reviews:
        print("No reviews found. Checking for CAPTCHA...")
        captcha_solution = solve_captcha()
        if captcha_solution:
            time.sleep(random.uniform(5, 10))  # Wait after solving CAPTCHA
            continue
        else:
            print("Could not solve CAPTCHA. Stopping the scraper.")
            break

    for review in reviews:
        try:
            reviewer = review.find_element(By.XPATH, ".//div[@class='middle']/span").text.strip()
        except:
            reviewer = "No Name"

        try:
            date = review.find_element(By.XPATH, ".//span[@class='title right']").text.strip()
        except:
            date = "No Date"

        try:
            content = review.find_element(By.XPATH, ".//div[@class='content']").text.strip()
        except:
            content = "No Review Content"

        reviews_data.append([reviewer, date, content])

    print(f"Page {page} scraped. Total reviews so far: {len(reviews_data)}")

    # Try navigating to the next page
    try:
        pagination_section = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/div')
        next_page_button = pagination_section.find_element(By.XPATH, f".//button[contains(@class, 'next-pagination-item') and text()='{page + 1}']")

        if next_page_button:
            print(f"Clicking page {page + 1} button...")

            # Scroll to and click the next page button
            driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
            time.sleep(random.uniform(3, 6))  # Random wait

            driver.execute_script("window.scrollBy(0, -100);")  # Adjust position to avoid overlap
            time.sleep(random.uniform(2, 4))

            # Try clicking using JavaScript to bypass UI blocks
            driver.execute_script("arguments[0].click();", next_page_button)
            time.sleep(random.uniform(5, 10))

            # Wait until the reviews update to confirm page change
            WebDriverWait(driver, 20).until(EC.staleness_of(reviews[0]))
        else:
            print("Next page button not found. Stopping scrape.")
            break

    except Exception as e:
        print(f"Error navigating to next page: {e}")
        break

# Close WebDriver
driver.quit()

# Save reviews to CSV file
df = pd.DataFrame(reviews_data, columns=["Reviewer Name", "Review Date", "Review Content"])
df.to_csv("lazada_reviews.csv", index=False, encoding="utf-8")

print(f"Scraping complete. {len(reviews_data)} reviews saved to lazada_reviews.csv")


Scraping Page 1...
Page 1 scraped. Total reviews so far: 5
Clicking page 2 button...
Scraping Page 2...
Page 2 scraped. Total reviews so far: 10
Clicking page 3 button...
Scraping Page 3...
Page 3 scraped. Total reviews so far: 15
Clicking page 4 button...
Scraping Page 4...
Page 4 scraped. Total reviews so far: 20
Clicking page 5 button...
Scraping Page 5...
Page 5 scraped. Total reviews so far: 25
Clicking page 6 button...
Scraping complete. 25 reviews saved to lazada_reviews.csv
